In [1]:
import tensorflow as tf
from __future__ import print_function
import collections
from collections import Counter
import string
import re
import json
import sys
import modeling
from run_squad import convert_examples_to_features, SquadExample, write_predictions, input_fn_builder, model_fn_builder, FLAGS, validate_flags_or_throw, FeatureWriter, RawResult, get_final_text, _get_best_indexes, _compute_softmax
import tokenization
from tqdm import tqdm
import os

In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS.bert_config_file = './checkpoints/chinese_L-12_H-768_A-12/bert_config.json'
FLAGS.vocab_file = './checkpoints/chinese_L-12_H-768_A-12/vocab.txt'
FLAGS.init_checkpoint = './checkpoints/CIPS-sogou-unfactoid/model.ckpt-99520'
FLAGS.output_dir = './checkpoints/CIPS-sogou-unfactoid/'
FLAGS.predict_file = './data/CIPS-sogou/valid.json'
FLAGS.do_predict = True
FLAGS.n_best_size = 10
FLAGS.max_answer_length = 256
FLAGS.doc_stride = 256
FLAGS.max_seq_length = 512

In [3]:

bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

validate_flags_or_throw(bert_config)

tf.gfile.MakeDirs(FLAGS.output_dir)

tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

train_examples = None
num_train_steps = None
num_warmup_steps = None

model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint=FLAGS.init_checkpoint,
        learning_rate=FLAGS.learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=FLAGS.use_tpu)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efdd843dd10>, '_model_dir': './checkpoints/CIPS-sogou-unfactoid/', '_protocol': None, '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': None, '_evaluation_master': '', '_eva

In [4]:
def read_cips_nonfactoid_examples(input_file, is_training=False):
    """Read a cips json file into a list of SquadExample."""
    input_data = []
    with tf.gfile.Open(input_file, "r") as reader:
        for line in reader:
            result = json.loads(line.strip())
            input_data.append(result)

    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    examples = []
    for entry in tqdm(input_data):
        qas_id = entry["query_id"]
        question_text = entry["query"]
        prev_is_whitespace = True
        paragraph_text = ""
        for _, paragraph in enumerate(entry["passages"]):
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True
            paragraph_text = ""
            paragraph_text = paragraph["passage_text"]
            from_passage = paragraph["passage_id"] # 选择使用的passage
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(c)
                    else:
                        doc_tokens[-1] += c
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)
            start_position = -1
            end_position = -1
            orig_answer_text = ""
            is_impossible = False
            example = SquadExample(
                qas_id='%s-%d' % (entry["query_id"], from_passage),
                question_text=question_text,
                doc_tokens=doc_tokens,
                orig_answer_text=orig_answer_text,
                start_position=start_position,
                end_position=end_position,
                is_impossible=is_impossible)
            examples.append(example)
    return examples

In [5]:
def write_predictions_multi(all_examples, all_features, all_results, n_best_size,
                            max_answer_length, do_lower_case, output_prediction_file,
                            output_nbest_file, output_null_log_odds_file):
    """Write final predictions to the json file and log-odds of null if needed."""
    tf.logging.info("Writing predictions to: %s" % (output_prediction_file))
    tf.logging.info("Writing nbest to: %s" % (output_nbest_file))

    example_index_to_features = collections.defaultdict(list)
    for feature in all_features:
        example_index_to_features[feature.example_index].append(feature)

    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result

    _PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "PrelimPrediction",
        ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

    all_predictions = collections.OrderedDict()
    _Prediction = collections.namedtuple("Prediction", ["predict_text", "from_passage", "logits"])
    all_nbest_json = collections.OrderedDict()
    scores_diff_json = collections.OrderedDict()
    
    last_logits = -10000
    last_query = -1
    for (example_index, example) in enumerate(tqdm(all_examples)):
        features = example_index_to_features[example_index]
        query_id = int(example.qas_id.split('-')[0])
        if query_id != last_query:
            last_logits = -10000
            last_query = query_id
        from_passage = int(example.qas_id.split('-')[1]) # 使用选择的passage 
        
        prelim_predictions = []
        # keep track of the minimum score of null start+end of position 0
        score_null = 1000000  # large and positive
        min_null_feature_index = 0  # the paragraph slice with min mull score
        null_start_logit = 0  # the start logit at the slice with min null score
        null_end_logit = 0  # the end logit at the slice with min null score
        for (feature_index, feature) in enumerate(features):
            result = unique_id_to_result[feature.unique_id]
            start_indexes = _get_best_indexes(result.start_logits, n_best_size)
            end_indexes = _get_best_indexes(result.end_logits, n_best_size)
            # if we could have irrelevant answers, get the min score of irrelevant
            if FLAGS.version_2_with_negative:
                feature_null_score = result.start_logits[0] + \
                    result.end_logits[0]
                if feature_null_score < score_null:
                    score_null = feature_null_score
                    min_null_feature_index = feature_index
                    null_start_logit = result.start_logits[0]
                    null_end_logit = result.end_logits[0]
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # We could hypothetically create invalid predictions, e.g., predict
                    # that the start of the span is in the question. We throw out all
                    # invalid predictions.
                    if start_index >= len(feature.tokens):
                        continue
                    if end_index >= len(feature.tokens):
                        continue
                    if start_index not in feature.token_to_orig_map:
                        continue
                    if end_index not in feature.token_to_orig_map:
                        continue
                    if not feature.token_is_max_context.get(start_index, False):
                        continue
                    if end_index < start_index:
                        continue
                    length = end_index - start_index + 1
                    if length > max_answer_length:
                        continue
                    prelim_predictions.append(
                        _PrelimPrediction(
                            feature_index=feature_index,
                            start_index=start_index,
                            end_index=end_index,
                            start_logit=result.start_logits[start_index],
                            end_logit=result.end_logits[end_index]))

        if FLAGS.version_2_with_negative:
            prelim_predictions.append(
                _PrelimPrediction(
                    feature_index=min_null_feature_index,
                    start_index=0,
                    end_index=0,
                    start_logit=null_start_logit,
                    end_logit=null_end_logit))
        prelim_predictions = sorted(
            prelim_predictions,
            key=lambda x: (x.start_logit + x.end_logit),
            reverse=True)

        _NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
            "NbestPrediction", ["text", "start_logit", "end_logit"])

        seen_predictions = {}
        nbest = []
        for pred in prelim_predictions:
            if len(nbest) >= n_best_size:
                break
            feature = features[pred.feature_index]
            if pred.start_index > 0:  # this is a non-null prediction
                tok_tokens = feature.tokens[pred.start_index:(
                    pred.end_index + 1)]
                orig_doc_start = feature.token_to_orig_map[pred.start_index]
                orig_doc_end = feature.token_to_orig_map[pred.end_index]
                orig_tokens = example.doc_tokens[orig_doc_start:(
                    orig_doc_end + 1)]
                tok_text = " ".join(tok_tokens)

                # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens)

                final_text = get_final_text(tok_text, orig_text, do_lower_case)
                if final_text in seen_predictions:
                    continue

                seen_predictions[final_text] = True
            else:
                final_text = ""
                seen_predictions[final_text] = True

            nbest.append(
                _NbestPrediction(
                    text=final_text,
                    start_logit=pred.start_logit,
                    end_logit=pred.end_logit))

        # if we didn't inlude the empty option in the n-best, inlcude it
        if FLAGS.version_2_with_negative:
            if "" not in seen_predictions:
                nbest.append(
                    _NbestPrediction(
                        text="", start_logit=null_start_logit,
                        end_logit=null_end_logit))
        # In very rare edge cases we could have no valid predictions. So we
        # just create a nonce prediction in this case to avoid failure.
        if not nbest:
            nbest.append(
                _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        assert len(nbest) >= 1

        total_scores = []
        best_non_null_entry = None
        for entry in nbest:
            total_scores.append(entry.start_logit + entry.end_logit)
            if not best_non_null_entry:
                if entry.text:
                    best_non_null_entry = entry

        probs = _compute_softmax(total_scores)

        nbest_json = []
        for (i, entry) in enumerate(nbest):
            output = collections.OrderedDict()
            output["text"] = entry.text
            output["probability"] = probs[i]
            output["start_logit"] = entry.start_logit
            output["end_logit"] = entry.end_logit
            nbest_json.append(output)

        assert len(nbest_json) >= 1
        
        
        # 根据不同的context，选出最符合的答案
        if not FLAGS.version_2_with_negative:
            if query_id in all_predictions.keys():
                if (nbest_json[0]["start_logit"] + nbest_json[0]["end_logit"]) > all_predictions[query_id].logits:
                    all_predictions[query_id] = _Prediction(predict_text=nbest_json[0]["text"],
                                                            from_passage=from_passage,
                                                            logits=nbest_json[0]["start_logit"] + nbest_json[0]["end_logit"])
            else:
                all_predictions[query_id] = _Prediction(predict_text=nbest_json[0]["text"], 
                                                        from_passage=from_passage, 
                                                        logits=nbest_json[0]["start_logit"] + nbest_json[0]["end_logit"])
                
        else:
            # predict "" iff the null score - the score of best non-null > threshold
            score_diff = score_null - best_non_null_entry.start_logit - (
                best_non_null_entry.end_logit)
            scores_diff_json[query_id] = score_diff
            if score_diff > FLAGS.null_score_diff_threshold:
                all_predictions[query_id] = ""
            else:
                all_predictions[query_id] = best_non_null_entry.text
        
        if query_id in all_nbest_json:
            all_nbest_json[query_id].extend(nbest_json)
        else:
            all_nbest_json[query_id] = nbest_json
    
    # 重构all_predictions以适合sogou-nonfactoid的评价格式
    with tf.gfile.GFile("./checkpoints/CIPS-sogou-unfactoid/predictions_sogou_style.json", "w") as writer:
        for query_id in all_predictions.keys():
            writer.write(json.dumps({"query_id": query_id, "answer_text":all_predictions[query_id].predict_text, "from_passage":all_predictions[query_id].from_passage}) + "\n")
    
    with tf.gfile.GFile(output_prediction_file, "w") as writer:
        writer.write(json.dumps(all_predictions, indent=4) + "\n")
    
    with tf.gfile.GFile(output_nbest_file, "w") as writer:
        writer.write(json.dumps(all_nbest_json, indent=4) + "\n")

    if FLAGS.version_2_with_negative:
        with tf.gfile.GFile(output_null_log_odds_file, "w") as writer:
            writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

In [6]:
#coding: utf-8
eval_examples = read_cips_nonfactoid_examples(
            input_file=FLAGS.predict_file, is_training=False)

eval_writer = FeatureWriter(
    filename=os.path.join(FLAGS.output_dir, "eval.tf_record"),
    is_training=False)
eval_features = []

def append_feature(feature):
    eval_features.append(feature)
    eval_writer.process_feature(feature)

convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=FLAGS.max_seq_length,
    doc_stride=FLAGS.doc_stride,
    max_query_length=FLAGS.max_query_length,
    is_training=False,
    output_fn=append_feature)
eval_writer.close()

print("***** Running predictions *****")
print("  Num orig examples = %d", len(eval_examples))
print("  Num split examples = %d", len(eval_features))
print("  Batch size = %d", FLAGS.predict_batch_size)

all_results = []

predict_input_fn = input_fn_builder(
    input_file=eval_writer.filename,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=False)

# If running eval on the TPU, you will need to specify the number of
# steps.
all_results = []
for result in estimator.predict(
        predict_input_fn, yield_single_examples=True):
    if len(all_results) % 1000 == 0:
        tf.logging.info("Processing example: %d" % (len(all_results)))
    unique_id = int(result["unique_ids"])
    start_logits = [float(x) for x in result["start_logits"].flat]
    end_logits = [float(x) for x in result["end_logits"].flat]
    all_results.append(
        RawResult(
            unique_id=unique_id,
            start_logits=start_logits,
            end_logits=end_logits))

output_prediction_file = os.path.join(
    FLAGS.output_dir, "predictions.json")
output_nbest_file = os.path.join(
    FLAGS.output_dir, "nbest_predictions.json")
output_null_log_odds_file = os.path.join(
    FLAGS.output_dir, "null_odds.json")

write_predictions_multi(eval_examples, eval_features, all_results,
                        FLAGS.n_best_size, FLAGS.max_answer_length,
                        FLAGS.do_lower_case, output_prediction_file,
                        output_nbest_file, output_null_log_odds_file)

  0%|          | 0/43779 [00:00<?, ?it/s]

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] 0 、 25 除 以 ( 5 ##x - 6 ) = 0 、 5 算 x = 多 少 [SEP] 因 为 在 ｘ = ０ 的 条 件 下 ， 由 约 分 都 可 得 到 ５ ##ｘ ／ ｘ ##＝ ##５ ， 所 以 ｘ 可 以 等 于 任 何 非 零 常 数 [SEP]
INFO:tensorflow:token_to_orig_map: 22:0 23:0 24:0 25:0 26:0 27:0 28:0 29:0 30:0 31:0 32:0 33:0 34:0 35:0 36:0 37:0 38:0 39:0 40:0 41:0 42:0 43:0 44:0 45:0 46:0 47:0 48:0 49:0 50:0 51:0 52:0 53:0 54:0 55:0 56:0 57:0 58:0 59:0
INFO:tensorflow:token_is_max_context: 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:True 53:True 54:True 55:True 56:True 57:True 58:True 59:True
INFO:tensorflow:input_ids: 101 121 510 8132 7370 809 113 126 8206 118 127 114 134 121 510 126 5050 166 134 1914 2208 102 1728 711

INFO:tensorflow:token_is_max_context: 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:True 53:True 54:True 55:True 56:True 57:True 58:True 59:True 60:True 61:True 62:True 63:True 64:True 65:True 66:True 67:True 68:True 69:True 70:True 71:True 72:True 73:True 74:True 75:True 76:True 77:True 78:True 79:True 80:True 81:True 82:True 83:True 84:True 85:True 86:True 87:True 88:True 89:True 90:True 91:True 92:True 93:True 94:True 95:True 96:True 97:True 98:True 99:True 100:True 101:True 102:True 103:True 104:True 105:True 106:True 107:True 108:True 109:True 110:True 111:True 112:True 113:True 114:True 115:True 116:True 117:True 118:True 119:True 120:True 121:True 122:True 123:True 124:True 125:True 126:True 127:True 128:True 129:True 130:True 131:True 132:True 133:True 134:True 135:True 136:True 137:T

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:input_ids: 101 121 510 8132 7370 809 113 126 8206 118 127 114 134 121 510 126 5050 166 134 1914 2208 102 9163 9414 12092 9484 8025 8031 9163 13047 12092 8025 8035 9484 13047 12092 8929 807 1057 5018 671 702 3175 4923 8030 10748 11267 8929 8020 8035 8022 8030 8025 12911 8021 8041 9414 8929 10748 11267 8929 12092 9414 10748 12092 9089 9484 8026 408 2347 4761 753 1039 671 3613 3175 4923 5299 8077 8031 13047 11267 9484 12166 12092 8929 10981 8025 8031 8024 8030 13047 8025 8075 12092 9089 10981 11267 9484 8079 4638 6237 4638 1469 3221 8030 8024 3724 8074 8024 8075 8024 8061 4638 966 4500 3175 4923 753 1121 3175 4923 671 8030 13047 8025 8075 8025 8031 13047 8025 8032 12166 12092 9089 10981 11267 9484 8025 8030 10981 11267 9089 8025 8074 8025 8033 12166 12092 10981 11267 9163 8025 8074 8025 8075 8025 8032 12166 8025 8061 12092 9163 8025 8020 8074 11267 12166 11267 10981 8021 8041 9163 11267 9484 12166 8025 8030 12092 9163 11267 9484 12166 8025 8037 12092 9484 12166 12166 12092

...

***** Running predictions *****
  Num orig examples = %d 43779
  Num split examples = %d 65282
  Batch size = %d 8
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:  name = unique_ids, shape = (?,)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT

INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder

INFO:tensorflow:Processing example: 43000
INFO:tensorflow:Processing example: 44000
INFO:tensorflow:Processing example: 45000
INFO:tensorflow:Processing example: 46000
INFO:tensorflow:Processing example: 47000
INFO:tensorflow:Processing example: 48000
INFO:tensorflow:Processing example: 49000
INFO:tensorflow:Processing example: 50000
INFO:tensorflow:Processing example: 51000
INFO:tensorflow:Processing example: 52000
INFO:tensorflow:Processing example: 53000
INFO:tensorflow:Processing example: 54000
INFO:tensorflow:Processing example: 55000
INFO:tensorflow:Processing example: 56000
INFO:tensorflow:Processing example: 57000
INFO:tensorflow:Processing example: 58000
INFO:tensorflow:Processing example: 59000
INFO:tensorflow:Processing example: 60000
INFO:tensorflow:Processing example: 61000
INFO:tensorflow:Processing example: 62000
INFO:tensorflow:Processing example: 63000
INFO:tensorflow:Processing example: 64000
INFO:tensorflow:Processing example: 65000
INFO:tensorflow:prediction_loop ma

100%|██████████| 43779/43779 [58:02<00:00, 13.35it/s]   


In [7]:
%run ./data/CIPS-sogou/eval.py ./data/CIPS-sogou/valid.json ./checkpoints/CIPS-sogou-unfactoid/predictions_sogou_style.json 

Total Cnt: 5000
average bleu-4:	0.4400
average rouge-l:	0.5106
